In [ ]:
# Example 24: doppler frequencies of currently visible satellites
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

import asyncio
import numpy as np
from scipy.fft import fft, ifft
import time
import gpslib_tutorial as gpslib                       
from rtlsdr import RtlSdr
import errno
from collections import deque
import signal
import linecache
import sys


# -------- floating point types used in arrays ---------
# 32-bit types overall ~1.4 times faster than 64-bit
# (factor for single multiplication ~2)

MY_FLOAT = np.float32
MY_COMPLEX = np.complex64

# ---global variables ----------

MEAS_RUNNING = True     
SMP_TIME = np.int64(0)  
                        
SWEEP_CORR_AVG = 4                                   

# configuration SDR
SDR_FREQCORR = 1    
SDR_CENTERFREQ = 1575.42e6 
SDR_GAIN = 50              
SDR_BANDWIDTH = 0          
SAMPLE_RATE = 2.048e6      

N_CYC = 32                 
CODE_SAMPLES = 2048        
NGPS = N_CYC*CODE_SAMPLES  
CORR_AVG = 8               
                           
SEC_TIME = np.linspace(1,NGPS,NGPS,endpoint=True,dtype=MY_FLOAT)/SAMPLE_RATE  

MIN_FREQ = -5000.0
MAX_FREQ = +5000.0
STEP_FREQ = 200   
CORR_MIN = 8      

SAT_ALL = list(range(2,33))

DATA_FILE = '../data/230914_data_24s.bin'
START_STREAM = 0 

LIVE_MEAS = True
MEAS_TIME = 1800      # in seconds

# ---- Following parameter may depend on computer and operating system ------

ASYNCIO_SLEEP_TIME = 1.e-4  # POSIX: should be > 0 

MAX_SAT = 8         
IT_SWEEP_ALL = 10           # no of iterations for finding satellites; 
                            # needed time must be < 32ms (N_CYC)


# ------- Exception handling & Output -------------------

def printException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'\
          .format(filename, lineno,line.strip(), exc_obj))

# ------- data buffer -----------------

MAXBUFSIZE = 16        
BUFFER = deque([],maxlen=MAXBUFSIZE)
NBUF = 0               
BUFSKIP = 0


def pushToBuffer(data):
    global BUFFER
    global NBUF
    global BUFSKIP

    if NBUF >= MAXBUFSIZE:
        BUFFER.clear()
        NBUF = 0
        BUFSKIP += MAXBUFSIZE
        
    BUFFER.append(data)
    NBUF += 1
    
    
def pullFromBuffer():
    global BUFFER
    global NBUF
    global BUFSKIP

    try:
        data = BUFFER.popleft()
        NBUF -= 1
        skip = BUFSKIP
        BUFSKIP = 0
    except IndexError:
        data = []
        skip = 0
    
    return data,skip


# ----------- Streaming live data ----------------        

async def streamLive():
    global MEAS_RUNNING

    sdr = RtlSdr()
    sdr.set_bias_tee(True)
    sdr.sample_rate = SAMPLE_RATE
    if SDR_FREQCORR != 0:
        sdr.freq_correction = SDR_FREQCORR    
    sdr.center_freq = SDR_CENTERFREQ   
    sdr.gain = SDR_GAIN 
    sdr.bandwidth = SDR_BANDWIDTH    

    sdrClosed = False
    measTimeout = False
    loop = asyncio.get_running_loop()
    start_time = loop.time()
    end_time = start_time+MEAS_TIME
    try:
        async for samples in sdr.stream(num_samples_or_bytes=NGPS, 
                                        format='samples'):
            pushToBuffer(samples)
            if loop.time()>end_time:                
                MEAS_RUNNING = False                    
                print('Timeout')
            if not MEAS_RUNNING:
                print('sdr to stop ..')
                await sdr.stop()
                print('sdr stopped')
                sdr.close()
                sdrClosed = True
                print('sdr closed')
    except:    
        print('Exception from sdr.stop')
    finally:
        MEAS_RUNNING = False
        if not sdrClosed:
            print('sdr to close ..')
            #sdr.close()        
            #print('sdr closed')
                    

# ----------- Streaming saved data (2 byte IQ) -----------


async def streamData():
    global MEAS_RUNNING
    k = 0
    try:
        statusMsg = ''
        measTimeout = False
        loop = asyncio.get_running_loop()
        start_time = loop.time()
        end_time = start_time+MEAS_TIME
        with open(DATA_FILE,'rb') as f1:
            while k < START_STREAM:
                byteData = np.fromfile(f1,dtype=np.uint16,count=NGPS)
                k += 1
            while MEAS_RUNNING:
                if NBUF < 1:
                    byteData = np.fromfile(f1,dtype=np.uint16,count=NGPS)
                    im,re = np.divmod(byteData,256)
                    samples = np.asarray(re+1j*im,dtype=np.complex64)/127.5\
                                         - (1+1j)
                    if len(samples)>0:
                        pushToBuffer(samples)
                    else:
                        MEAS_RUNNING = False
                        statusMsg = 'EOF'
                    if (loop.time()>end_time):                
                        statusMsg = 'Timeout'
                        MEAS_RUNNING = False
                await asyncio.sleep(ASYNCIO_SLEEP_TIME)
    except BaseException as err:
        printException()
        statusMsg = err
    finally:
        MEAS_RUNNING = False
        print(statusMsg)
             
    
# ------- Find maximum in correlation (code phase) ---------

def findCodePhase(gpsCorr):
    mean = np.mean(gpsCorr)
    std = np.std(gpsCorr)
    delay = -1
    
    mx = np.argmax(gpsCorr)
    normMaxCorr = (gpsCorr[mx]-mean)/std
    if normMaxCorr > CORR_MIN:
        delay = mx
             
    return delay,normMaxCorr
    
            
# ------  demodulate doppler frequency, used for sweepFrequency ---------

def demodDoppler(data,dopplerFreq,dopplerPhase,N):                          
    factor = np.exp(-1j*(dopplerPhase+2*np.pi*dopplerFreq*SEC_TIME[:N]))
    dopplerPhase += 2*np.pi*dopplerFreq*SEC_TIME[N-1]
    return factor*data[:N], np.remainder(dopplerPhase,2*np.pi)


# -------- sweep frequency -------------
 

def sweepAllSats(data,freq,satLst,satFound,itSweep=2):   
    sweepReady = False
    avg = min(SWEEP_CORR_AVG,N_CYC)
    N = avg*CODE_SAMPLES
    
    phase = 0
    it = 0
    while freq < MAX_FREQ and it < itSweep:
        newData,_ = demodDoppler(data,freq,phase,N)    
        df = 0
        for i in range(avg):
            dfm = fft(newData[i*CODE_SAMPLES:(i+1)*CODE_SAMPLES]) 
            df += dfm
        fftData = df/avg

        removeLst = []
        for satNo in satLst: 
            corr = np.abs(ifft(fftData*np.conjugate(FFT_CACODE[satNo])))
            delay,normMaxCorr = findCodePhase(corr)
            if delay > -1:
                satFound.append((normMaxCorr,satNo,freq,delay))
                removeLst.append(satNo)

        for satNo in removeLst:
            satLst.remove(satNo)
        
        freq += STEP_FREQ
        if (freq >= MAX_FREQ):
            sweepReady = True
            freq -= MAX_FREQ - MIN_FREQ
            
        it += 1
        
    return sweepReady,freq,sorted(satFound,reverse=True)
             

def reportFoundSats(foundSats):
    print('Found Satellites:')
    for fs in foundSats:
        print(f'PRN {fs[1]:02d} Corr:{fs[0]:4.1f}  f={fs[2]:+7.1f}')
    print()


# ------- main - process data  -------------------

async def processData():   
    global MEAS_RUNNING
    global SMP_TIME

    sweepAllFreq = True
    dopplerFreq = MIN_FREQ              # start frequency for sweep 

    skippedData = 0
    satLst = SAT_ALL.copy()
    #satLst = [13,22,24,25,32] #24
    #satLst = [12,17,25,24,19,15,13,10,22,32]
    foundSats = []
    ts = None
    try:
        while MEAS_RUNNING:

            if NBUF >= 1:
                if ts is None:
                    ts = time.time()
                data,skip = pullFromBuffer()
                skippedData += skip         
                SMP_TIME += (1 + skip)*NGPS 
                
                if sweepAllFreq:                 
                    t1 = time.perf_counter()
                    sweepReady,dopplerFreq,foundSats \
                        = sweepAllSats(data,dopplerFreq,satLst,\
                                       foundSats,itSweep=IT_SWEEP_ALL)
                    t2 = time.perf_counter()
                    T.append(t2-t1)
                    if sweepReady:
                        te = time.time()
                        reportFoundSats(foundSats) 
                        print('IT_SWEEP_ALL: %d' % (IT_SWEEP_ALL))
                        print('skipped data: %d (%d)' % (skippedData,NBUF))
                        print('number of calls: %d' % (len(T)))
                        print('total time: %1.3f s' % (te-ts))
                        print('total time of calls: %1.3f s' % (np.sum(T)))
                        print('mean time of calls: %1.3f ms' % (np.mean(T)*1000))
                        print('max time of calls: %1.3f ms' % (np.max(T)*1000))
                        print()
                        MEAS_RUNNING = False
                        
            await asyncio.sleep(ASYNCIO_SLEEP_TIME)
            
    except BaseException as err:
        printException()
    finally:            
        MEAS_RUNNING = False    

            
    
# --------------- Main ------------------      

FFT_CACODE = [0,0]  
for sat in SAT_ALL: 
    FFT_CACODE.append(fft(gpslib.GPSCacode(sat,CODE_SAMPLES)))

loop = asyncio.get_event_loop()
task2 = loop.create_task(processData())
if LIVE_MEAS:
    task1 = loop.create_task(streamLive())
else:
    task1 = loop.create_task(streamData())
